### requirements.txt

In [62]:
%%writefile requirements.txt
# YOLOv5 requirements
# Usage: pip install -r requirements.txt

# Base ------------------------------------------------------------------------
gitpython>=3.1.30
matplotlib>=3.3
numpy>=1.22.2
opencv-python>=4.1.1
Pillow>=10.0.1
psutil  # system resources
PyYAML>=5.3.1
requests>=2.23.0
scipy>=1.4.1
thop>=0.1.1  # FLOPs computation
torch>=1.8.0  # see https://pytorch.org/get-started/locally (recommended)
torchvision>=0.9.0
tqdm>=4.64.0
ultralytics>=8.0.147
# protobuf<=3.20.1  # https://github.com/ultralytics/yolov5/issues/8012

# Logging ---------------------------------------------------------------------
# tensorboard>=2.4.1
# clearml>=1.2.0
# comet

# Plotting --------------------------------------------------------------------
pandas>=1.1.4
seaborn>=0.11.0

# Export ----------------------------------------------------------------------
# coremltools>=6.0  # CoreML export
# onnx>=1.10.0  # ONNX export
# onnx-simplifier>=0.4.1  # ONNX simplifier
# nvidia-pyindex  # TensorRT export
# nvidia-tensorrt  # TensorRT export
# scikit-learn<=1.1.2  # CoreML quantization
# tensorflow>=2.4.0  # TF exports (-cpu, -aarch64, -macos)
# tensorflowjs>=3.9.0  # TF.js export
# openvino-dev>=2023.0  # OpenVINO export

# Deploy ----------------------------------------------------------------------
setuptools>=65.5.1 # Snyk vulnerability fix
# tritonclient[all]~=2.24.0

# Extras ----------------------------------------------------------------------
# ipython  # interactive notebook
# mss  # screenshots
# albumentations>=1.0.3
# pycocotools>=2.0.6  # COCO mAP

Overwriting requirements.txt


### descargar archivos

In [63]:
!gdown 1_hVkSGea_iqTdJznAF5ZbnR8aoa-J86-

Downloading...
From: https://drive.google.com/uc?id=1_hVkSGea_iqTdJznAF5ZbnR8aoa-J86-
To: /content/yolov5/people.png
100% 4.08M/4.08M [00:00<00:00, 216MB/s]


# Setup
Clonamos el repositorio e instalamos requisitos.txt en un entorno Python>=3.8.0, incluido PyTorch>=1.8.

In [64]:
!git clone https://github.com/ultralytics/yolov5  # clone
%cd yolov5
%pip install -qr requirements.txt comet_ml  # install

import torch
import utils
display = utils.notebook_init()  # checks

YOLOv5 🚀 v7.0-247-g3f02fde Python-3.10.12 torch-2.1.0+cu118 CPU


Setup complete ✅ (2 CPUs, 12.7 GB RAM, 29.5/107.7 GB disk)


# 1. Detect
detect.py ejecuta inferencia en una variedad de fuentes, descarga modelos automáticamente desde la última versión de YOLOv5 y guarda los resultados para ejecutar/detectar.

In [65]:
import io
import csv
from contextlib import redirect_stdout

# Captura la salida de la consola
output_buffer = io.StringIO()

# Redirige stdout a la variable
with redirect_stdout(output_buffer):

    !python detect.py --weights yolov5s.pt --source 'https://youtu.be/d4xmdH2NKRI'

# Obtiene la salida capturada como una cadena
output_string = output_buffer.getvalue()

# Guarda la salida en un archivo CSV
with open('output.csv', 'w', newline='') as csvfile:
    csv_writer = csv.writer(csvfile, delimiter=',')

    # Divide la salida en líneas y escribe cada línea en el archivo CSV
    for line in output_string.split('\n'):
        csv_writer.writerow(line.split())

print(output_string)

detect: weights=['yolov5s.pt'], source=https://youtu.be/d4xmdH2NKRI, data=data/coco128.yaml, imgsz=[640, 640], conf_thres=0.25, iou_thres=0.45, max_det=1000, device=, view_img=False, save_txt=False, save_csv=False, save_conf=False, save_crop=False, nosave=False, classes=None, agnostic_nms=False, augment=False, visualize=False, update=False, project=runs/detect, name=exp, exist_ok=False, line_thickness=3, hide_labels=False, hide_conf=False, half=False, dnn=False, vid_stride=1
YOLOv5 🚀 v7.0-247-g3f02fde Python-3.10.12 torch-2.1.0+cu118 CPU

100% 14.1M/14.1M [00:00<00:00, 106MB/s] 

Fusing layers... 
YOLOv5s summary: 213 layers, 7225885 parameters, 0 gradients, 16.4 GFLOPs
WARNING ⚠️ Environment does not support cv2.imshow() or PIL Image.show()

1/1: https://youtu.be/d4xmdH2NKRI...  Success (3599 frames 1280x720 at 30.00 FPS)

0: 384x640 17 persons, 536.5ms
0: 384x640 15 persons, 433.5ms
0: 384x640 14 persons, 423.6ms
0: 384x640 16 persons, 424.7ms
0: 384x640 18 persons, 434.5ms
0: 384x64

# 2. Extracción y análisis CSV
Se guarda en un csv separado  po rcolum

In [67]:
import re
# Extraer líneas relevantes de la salida
lines = output_string.split('\n')
data_lines = [line.strip() for line in lines if line.startswith('0:')]

# Crea un archivo CSV
with open('output2.csv', 'w', newline='') as csvfile:
    csv_writer = csv.writer(csvfile)

    # Escribir el encabezado
    csv_writer.writerow(['Frame', 'Resolution', 'Objects', 'Time'])

    # Escribir datos en el archivo CSV
    for line in data_lines:
        parts = re.split(r'\s+', line)
        frame = parts[0][:-1]  # Eliminar el ':' al final del número de fotograma
        resolution = parts[1]
        objects = ' '.join(parts[2:-1])  # Unir las palabras restantes como objetos
        time = parts[-1][:-2]  # Eliminar 'ms' al final del tiempo

        # Escribir la fila en el archivo CSV
        csv_writer.writerow([frame, resolution, objects, time])

print("Archivo CSV creado con éxito.")

Archivo CSV creado con éxito.


# Promedio de objectos en el video total

In [68]:
# Extraer líneas relevantes de la salida
lines = output_string.split('\n')
data_lines = [line.strip() for line in lines if line.startswith('0:')]

# Contadores para el promedio
persons_count = 0
potted_plants_count = 0
bench_count = 0
backpack_count = 0
total_frames = 0

for line in data_lines:
    parts = re.split(r'\s+', line)
    objects = ' '.join(parts[2:-1])  # Unir las palabras restantes como objetos

    if 'persons' in objects:
        # Buscar un número seguido de "persons"
        persons_match = re.search(r'(\d+)', objects)
        persons_count += int(persons_match.group()) if persons_match else 0

    if 'potted plant' in objects:
        # Buscar un número seguido de "potted plant"
        potted_plants_match = re.search(r'(\d+)', objects)
        potted_plants_count += int(potted_plants_match.group(1)) if potted_plants_match else 0

    if 'bench' in objects:
        # Buscar un número seguido de "bench" o "benchs"
        bench_match = re.search(r'(\d+)', objects)
        bench_count += int(bench_match.group(1)) if bench_match else 0

    if 'backpack' in objects:
        # Buscar un número seguido de "backpack"
        backpack_match = re.search(r'(\d+)', objects)
        backpack_count += int(backpack_match.group(1)) if backpack_match else 0

    total_frames += 1

# Calcula el promedio
average_persons = persons_count / total_frames
average_potted_plants = potted_plants_count / total_frames
average_bench = bench_count / total_frames
average_backpack = backpack_count / total_frames

# los resultados
print(f"Promedio de personas captadas por frame: {average_persons}")
print(f"Promedio de plantas en maceta captadas por frame: {average_potted_plants}")
print(f"Promedio de bancos captados por frame: {average_bench}")
print(f"Promedio de mochilas captadas por frame: {average_backpack}")

Promedio de personas captadas por frame: 15.344444444444445
Promedio de plantas en maceta captadas por frame: 5.197076023391813
Promedio de bancos captados por frame: 0.7964912280701755
Promedio de mochilas captadas por frame: 0.07894736842105263


# Ordenar por tiempo

In [69]:
# Estructura de datos para almacenar la información
data_list = []

# Extraer líneas relevantes de la salida
lines = output_string.split('\n')
data_lines = [line.strip() for line in lines if line.startswith('0:')]

# Procesar cada línea y almacenar la información en la estructura de datos
for line in data_lines:
    parts = re.split(r'\s+', line)
    objects = ' '.join(parts[2:-1])  # Unir las palabras restantes como objetos
    time_ms = float(parts[-1][:-2])  # Extraer el tiempo en milisegundos y convertir a float

    # Almacenar la información en un diccionario
    data_list.append({
        'objects': objects,
        'time_ms': time_ms
    })

# Filtrar y ordenar la lista según tus criterios (en este caso, 'persons')
filtered_data = [item for item in data_list if 'persons' in item['objects']]
sorted_data = sorted(filtered_data, key=lambda x: x['time_ms'])

# resultados
for item in sorted_data:
    print(f"Objects: {item['objects']}, Time (ms): {item['time_ms']}")

Objects: 12 persons,, Time (ms): 254.9
Objects: 19 persons, 1 bench,, Time (ms): 255.0
Objects: 13 persons, 1 potted plant,, Time (ms): 255.9
Objects: 15 persons,, Time (ms): 256.1
Objects: 15 persons, 2 potted plants,, Time (ms): 256.9
Objects: 19 persons, 1 bench, 1 potted plant,, Time (ms): 257.2
Objects: 16 persons, 1 potted plant,, Time (ms): 257.3
Objects: 16 persons,, Time (ms): 257.4
Objects: 16 persons, 1 horse, 3 potted plants,, Time (ms): 257.4
Objects: 13 persons,, Time (ms): 257.6
Objects: 17 persons,, Time (ms): 257.7
Objects: 17 persons,, Time (ms): 258.0
Objects: 20 persons,, Time (ms): 258.0
Objects: 15 persons,, Time (ms): 258.1
Objects: 14 persons,, Time (ms): 258.1
Objects: 14 persons,, Time (ms): 258.4
Objects: 13 persons, 1 potted plant,, Time (ms): 258.4
Objects: 12 persons,, Time (ms): 258.4
Objects: 15 persons,, Time (ms): 258.5
Objects: 12 persons,, Time (ms): 258.5
Objects: 16 persons,, Time (ms): 258.7
Objects: 16 persons,, Time (ms): 258.8
Objects: 22 perso

# Objetos en cierto rango de tiempo

In [70]:
# Solicita al usuario que ingrese el rango de tiempo
start_time = float(input("Ingrese el tiempo de inicio [254 como min(ms)]: "))
end_time = float(input("Ingrese el tiempo de finalización [638(ms)]: "))

# Filtra la lista según el rango de tiempo
filtered_data = [item for item in data_list if start_time <= item['time_ms'] <= end_time]

# resultados
print("Objetos que aparecen en cierto rango de tiempo:")
for item in filtered_data:
    print(f"{item['objects']}")

Ingrese el tiempo de inicio [254 como min(ms)]: 254
Ingrese el tiempo de finalización [638(ms)]: 262
Objetos que aparecen en cierto rango de tiempo:
21 persons,
15 persons, 1 potted plant,
21 persons, 1 bench, 1 potted plant,
19 persons,
19 persons, 1 bench, 1 potted plant,
19 persons, 1 bench,
16 persons,
16 persons,
12 persons, 1 bench,
15 persons,
15 persons,
17 persons,
17 persons,
17 persons,
15 persons,
14 persons,
16 persons,
16 persons,
15 persons,
16 persons,
15 persons,
14 persons, 1 skateboard, 1 potted plant,
16 persons, 1 potted plant,
19 persons, 1 bench, 1 potted plant,
16 persons, 1 motorcycle, 1 handbag,
15 persons, 2 potted plants,
19 persons,
15 persons,
15 persons,
16 persons, 1 horse, 3 potted plants,
19 persons,
18 persons, 1 bicycle,
20 persons,
17 persons, 2 potted plants,
16 persons, 1 potted plant,
16 persons, 1 potted plant,
19 persons,
18 persons,
13 persons,
13 persons,
15 persons,
14 persons,
13 persons,
12 persons,
15 persons,
15 persons,
12 persons,
12 p